<h1 style="text-align:center;"><a href="https://docs.wandb.ai/" target="_blank">Weights and Biases(wandb)</a></h1>
<p style="text-align:center;">模型训练可视化分析<p>
<p style="text-align:center;font-size:18px;">2024-07-09</p>

### 特性

* 日志上传云端永久存储，便于分享不怕丢失。

* 可以存管代码,数据集和模型的版本，随时复现。(wandb.Artifact)

* 可以使用交互式表格进行case分析(wandb.Table)

* 可以自动化模型调参。(wandb.sweep)

### 核心功能

1. 实验跟踪：experiment tracking （wandb.log）

2. 版本管理：version management (wandb.log_artifact, wandb.save)

3. case分析：case visualization (wandb.Table, wandb.Image)

4. 超参调优：model optimization (wandb.sweep)

#### 安装

In [1]:
# !pip install wandb

#### <a href="https://wandb.ai/login" target="_blank">注册</a>, 右上角中找到Quickstart，获取API Key
<div style="text-align:center"><img src="image/index/1719988297636.png" width="500px" style="border-radius: 10px;border: 1px solid #ddd;"></div>
<div style="text-align:center"><img src="image/index/1719988151134.png" width="500px" style="border-radius: 10px;border: 1px solid #ddd;"></div>

#### 登陆
* <p style="font-size:20px">设置环境变量: <strong>WANDB_API_KEY=@your_api_key</strong>(建议写在.env文件中)</p>

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

import wandb
wandb.login(key=os.getenv("WANDB_API_KEY"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: reviy (esil). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hehaowen/.netrc


True

#### 设置参数

In [3]:
import torch
from argparse import Namespace

config = Namespace(
    project_name = "wandb_esc10_demo", # wandb项目名称
    data_dir = 'ESC-50-master/audio', # 音频文件目录
    meta_file = 'ESC-50-master/meta/esc50.csv', # 元数据文件
    sr = 22050, # 采样率
    duration = 5, # 音频时长
    epochs = 10, # 训练轮数
    batch_size = 32, # 批次大小
    lr = 0.001, # 学习率
    random_seed = 1202, # 随机种子
    n_fft = 1024, # FFT的窗长
    hop_length = 512, # 窗步长
    n_mels = 64, # 梅尔滤波器组的数量
    dropout = 0.1, # 丢弃率
)
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 选择设备
device

'cpu'

#### 读取元数据文件

In [4]:
import pandas as pd
df = pd.read_csv(config.meta_file)
df = df[df['esc10']==True] # 只选取esc10的音频
categoties = df['category'].unique() # 10个类别
label_df = pd.DataFrame({'category': categoties})
label_df.T

,0,1,2,3,4,5,6,7,8,9
category,dog,chainsaw,crackling_fire,helicopter,rain,crying_baby,clock_tick,sneezing,rooster,sea_waves


In [5]:
df['label'] = df['category'].apply(lambda x: categoties.tolist().index(x))
df.head(3)

,filename,fold,target,category,esc10,src_file,take,label
0,1-100032-A-0.wav,1,0,dog,True,100032,A,0
14,1-110389-A-0.wav,1,0,dog,True,110389,A,0
24,1-116765-A-41.wav,1,41,chainsaw,True,116765,A,1


#### 拆分训练集和测试集

In [6]:
from sklearn.model_selection import train_test_split

X, y = df['filename'].values, df['label'].values 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=config.random_seed)
len(X_train), len(X_test), len(y_train), len(y_test)

(320, 80, 320, 80)

#### 构建数据集

In [7]:
from torchaudio.transforms import MelSpectrogram
from dataset import AudioDataset

dataset_params = {
    'data_dir': config.data_dir,
    'sr': config.sr,
    'duration': config.duration,
    'device': device,
    'transform': MelSpectrogram(
        sample_rate=config.sr,
        n_fft=config.n_fft,
        hop_length=config.hop_length,
        n_mels=config.n_mels
    )
}
train_dataset = AudioDataset(X=X_train, y=y_train, **dataset_params)
test_dataset = AudioDataset(X=X_test, y=y_test, **dataset_params)
train_dataset[0][0].shape

torch.Size([1, 64, 216])

#### 创建Dataloaders

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
for _, (data, target) in enumerate(train_dataloader):
    data_shape, target_shape = data.shape, target.shape
    break
data_shape, target_shape

(torch.Size([32, 1, 64, 216]), torch.Size([32]))

#### 创建模型

In [9]:
from models.panns import CNN10
model = CNN10(num_class=len(categoties), input_size=data_shape[-1], dropout=config.dropout)
model.to(device)

CNN10(num_class=10, input_size=216, dropout=0.1)

#### 创建optimizer, loss function

In [10]:
from torch import nn
optimize = torch.optim.Adam(model.parameters(), lr=config.lr)
loss_func = nn.CrossEntropyLoss().to(device)

#### 定义训练方法

In [11]:
def train_an_epoch(model, optimizer, data_loader, device, loss_func):
    model.train()
    running_loss = 0.0
    for batch_idx, (features, labels) in enumerate(data_loader):
        features = features.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(data_loader)

#### 定义测试方法

In [12]:
def test_an_epoch(model, data_loader, device, loss_func):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (features, labels) in enumerate(data_loader):
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            loss = loss_func(outputs, labels)
            total_loss += loss.item()
            outputs = torch.softmax(outputs, dim=1)